In [1]:
from pymongo import MongoClient
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll


db, coll = connectCollection('companies','companies')
db, offices = connectCollection('companies', 'offices')
# db.create_collection("offices")
all_comps = list(coll.find())
all_offices = list(offices.find())

In [83]:
offices[0]

{'_id': ObjectId('5dd13bf6c80571d59fb0a390'),
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-122.394523, 37.764726]},
 'properties': {'name': 'Digg',
  'country': 'USA',
  'company_money_raised': '$45M',
  'foundation_year': 2004}}

In [41]:
target_offices = list(offices.find({"$and" : [{"properties.country":"GBR"}, {"properties.foundation_year": {"$gt": 2009}}, {"properties.company_money_raised" : {"$gte" :"$1M"}} ]}))
geometry = target_offices[0]['geometry']
target_offices[0]

{'_id': ObjectId('5dd13bf7c80571d59fb0ac42'),
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-0.208979, 51.51819]},
 'properties': {'name': 'Pixsta',
  'country': 'GBR',
  'company_money_raised': '£3M',
  'foundation_year': 2010}}

In [39]:
def target_offices(country_code, foundation_year, money_raised):
        return list(offices.find({
            "$and" : [{
                "properties.country":f"{country_code}"}, {
                    "properties.foundation_year": {"$gt": foundation_year}}, {
                        "properties.company_money_raised" : {"$gte" :f"${money_raised}M"}
                        }]
                        } ))
target_offices("GBR", 2008, 1)

[{'_id': ObjectId('5dd13bf7c80571d59fb0ac42'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.208979, 51.51819]},
  'properties': {'name': 'Pixsta',
   'country': 'GBR',
   'company_money_raised': '£3M',
   'foundation_year': 2010}},
 {'_id': ObjectId('5dd13bf7c80571d59fb0b2c3'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-1.574948, 54.703145]},
  'properties': {'name': 'CompareAway',
   'country': 'GBR',
   'company_money_raised': '£500k',
   'foundation_year': 2009}},
 {'_id': ObjectId('5dd13bf9c80571d59fb0c015'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.1020265, 51.5196135]},
  'properties': {'name': 'Shutl',
   'country': 'GBR',
   'company_money_raised': '£7.66M',
   'foundation_year': 2009}}]

In [42]:
def near_offices(geometry, max_distance=2000):
    return list(offices.find({"geometry.coordinates": 
        {"$near" : 
             {
            "$geometry": geometry ,
            "$maxDistance": max_distance
        }}
    }))
near_offices(geometry)

[{'_id': ObjectId('5dd13bf7c80571d59fb0ac42'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.208979, 51.51819]},
  'properties': {'name': 'Pixsta',
   'country': 'GBR',
   'company_money_raised': '£3M',
   'foundation_year': 2010}},
 {'_id': ObjectId('5dd13bf6c80571d59fb0a44d'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.205349, 51.514935]},
  'properties': {'name': 'ConnectMeAnywhere',
   'country': 'GBR',
   'company_money_raised': '$0',
   'foundation_year': 2006}},
 {'_id': ObjectId('5dd13bf7c80571d59fb0ab55'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.209019, 51.5260475]},
  'properties': {'name': 'SOSIUS',
   'country': 'GBR',
   'company_money_raised': '$0',
   'foundation_year': 2007}},
 {'_id': ObjectId('5dd13bf9c80571d59fb0bfa5'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.1855258, 51.5174054]},
  'properties': {'name': 'Ruby50',
   'country': 'GBR',
   'company_m

In [146]:
def old_companies(offices, year):
    old_companies = []
    for office in offices: 
        if office['properties']['foundation_year'] < year:
            old_companies.append(office)
    return old_companies

old_companies(near_offices(geometry), 2006)

[]

### ------------

In [129]:
loc

{}

In [111]:
def geocode_request(loc):
    for e in comps:
        for office in e['offices']:
            latitude = office['latitude']
            longitude = office['longitude']
            if latitude == True and longitude == True: 
                url = f"https://geocode.xyz/{latitude},{logitude}?geoit=JSON"
            res = requests.get(url)
                if res.status_code == 200:
                    print(res.url)
            res.json()
            return (data.get("latt",0), data.get("longt",0))
        else:
            raise ValueError(f"Api returned {res.status_code}!=200")

In [112]:
getCoordsFor(coords[0])

https://geocode.xyz/(51.51819,%20-0.208979)?geoit=JSON


('0.00000', '0.00000')

In [106]:
import requests
import folium

# TODO: Center the map with the midde point of all coordinates
# https://fiona.readthedocs.io/en/latest/manual.html
def plotCoords(lat, long):
    marker_coords = []
    marker_coords.append(getCoordsFor(lat, long)) 
    m = folium.Map()
    folium.Marker(lat, long).add_to(m)
    return m

In [1]:
import src.companiesdb as comp
import src.google_api_requests
db, offices = comp.connectCollection('companies', 'offices')

In [2]:
target = comp.target_offices(offices, 'USA', 2009, 1)
near_count = []
old_count = []
tech_startups_count = []

In [3]:
result = []
for e in target_companies:
    near = comp.near_offices(offices, e['geometry'])
    near_count.append(len(near))
    old = comp.old_companies(near, 2003)
    old_count.append(len(old))
    tech_startups = comp.tech_startups(near, 2009, target)
    tech_startups_count.append(len(tech_startups))
    res = {
        'reference' : e['properties']['name'],
        'near_companies' : near,
        'old_companies' : old,
        'tech_startups' : tech_startups
    }
    result.append(res)
df = pd.Dataframe({
    'Name' : names,
    'Companies Close' : near_count,
    'Corporations Close' : old_count,
    'Tech Startups' : tech_startups_count
})

In [4]:
print(len(target), len(result), len(near_count), len(old_count), len(tech_startups_count))

26 26 26 26 26


In [5]:
result

[{'reference': 'GENWI',
  'near_companies': [{'_id': ObjectId('5dd42889da6c525e8be7c6fa'),
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-111.9035, 33.8171]},
    'properties': {'name': 'GENWI',
     'country': 'USA',
     'company_money_raised': '$7.1M',
     'foundation_year': 2010,
     'category': 'mobile'}}],
  'old_companies': [],
  'tech_startups': [{'_id': ObjectId('5dd42889da6c525e8be7c6fa'),
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-111.9035, 33.8171]},
    'properties': {'name': 'GENWI',
     'country': 'USA',
     'company_money_raised': '$7.1M',
     'foundation_year': 2010,
     'category': 'mobile'}}]},
 {'reference': 'Fixya',
  'near_companies': [{'_id': ObjectId('5dd42889da6c525e8be7c734'),
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-122.323895, 37.566879]},
    'properties': {'name': 'Fixya',
     'country': 'USA',
     'company_money_raised': '$8M',
     'foundation_year': 2013,


In [8]:
target[0]['properties']['name']

'GENWI'

In [ ]:
pd.Dataframe({'Name' : names
    
})